In [5]:
import requests
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET
from kafka import KafkaProducer
import json
import re

# Kafka configuration
KAFKA_TOPIC = "vnexpress_rss"
KAFKA_BOOTSTRAP_SERVERS = "localhost:9092"

def fetch_rss_data(rss_url):
    response = requests.get(rss_url)
    response.raise_for_status()
    return response.content

def parse_rss_data(rss_content):
    root = ET.fromstring(rss_content)
    items = root.findall(".//item")
    articles = []
    for item in items:
        # Lấy nội dung
        description = item.find("description").text
        clean_description = extract_clean_text(description)

        article = {
            "title": item.find("title").text,
            "pubDate": item.find("pubDate").text,
            "link": item.find("link").text,
            "description": clean_description,
        }
        articles.append(article)
    return articles

def extract_clean_text(html_text):
    # Trích xuất nội dung văn bản
    if html_text:
        soup = BeautifulSoup(html_text, "html.parser")
        return soup.get_text().strip()
    return "No description available"

def fetch_author_from_article(link):
    # Lấy thông tin tác giả từ bài viết.
    try:
        response = requests.get(link)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Tìm thẻ <p class="author_mail"> hoặc các thẻ chứa tác giả khác
        author_tag = soup.find("p", class_="author_mail")
        if author_tag and author_tag.strong:
            return author_tag.strong.text.strip()
        return "Unknown"
    except Exception as e:
        print(f"Error fetching author from {link}: {e}")
        return "Unknown"

def send_to_kafka(producer, topic, articles):
    for article in articles:
        # Lấy thông tin tác giả
        author = fetch_author_from_article(article["link"])
        article["author"] = author
        
        # Gửi dữ liệu lên Kafka
        producer.send(topic, value=article)
        print(f"Sent to Kafka: {article['title']}")

if __name__ == "__main__":
    rss_url = "https://vnexpress.net/rss/thoi-su.rss"
    kafka_producer = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        value_serializer=lambda v: json.dumps(v, ensure_ascii=False).encode("utf-8")
    )
    
    try:
        rss_content = fetch_rss_data(rss_url)
        articles = parse_rss_data(rss_content)
        send_to_kafka(kafka_producer, KAFKA_TOPIC, articles)
    finally:
        kafka_producer.close()

Sent to Kafka: Nhân tài được vay ưu đãi mua nhà trả góp
Sent to Kafka: 'Mở cửa' cho tích tụ đất đai trong nông nghiệp
Sent to Kafka: Biển người ở trung tâm Hà Nội, TP HCM đón năm mới
Sent to Kafka: Tổng rà soát chất lượng cán bộ, công chức, viên chức
Sent to Kafka: Đề xuất chính sách đặc thù cho dự án điện hạt nhân Ninh Thuận
Sent to Kafka: Cấm xe 20 đường trung tâm TP HCM để bắn pháo hoa
Sent to Kafka: Metro chạy xuyên đêm chở khách đón Tết Dương lịch
Sent to Kafka: Ôtô tông vào rạp đám tang dựng dưới lòng đường
Sent to Kafka: 68 cán bộ diện Trung ương quản lý bị kỷ luật trong năm 2024
Sent to Kafka: Học viện Chính trị quốc gia Hồ Chí Minh giảm 23 đầu mối
Sent to Kafka: Bạc Liêu đầu tư 1.450 tỷ đồng làm đường vành đai
Sent to Kafka: Tàu chở 4 thuyền viên mất tích trên biển
Sent to Kafka: Phát hiện dấu tích thành cổ Hoa Lư khi đào móng nhà
Sent to Kafka: Sập giàn giáo công trình thủy điện, 3 người chết
Sent to Kafka: Nữ nhân viên đường sắt bị đánh gãy xương mũi vì 'không mở gác chắn tà